# Model stodoła

That means aggregating to whole parts

Data reqiured are sieckm_pot_nod.csv and DaneSIM.csv (later in code can be changed)

In [1]:
import csv

In [2]:
import sqlite3

In [3]:
with open("sieckm_pot_nod.csv",'r') as cvsfile:
    f=csv.DictReader(cvsfile)
    fieldnames=f.fieldnames
    csvf=[r for r in f]

csvf is a list of dictionaries [{fieldname:value,...},{...}]. This contains elements in each ring. [{Id:1 , Ring0:"13 16" ,"Ring1":"50 10 11".....

In [4]:
#dbconnbkp=sqlite3.connect('stodola.db')
dbconn=sqlite3.connect(':memory:')

In [5]:
c=dbconn.cursor()

In [6]:
c.execute('''drop table if exists model''')

In [7]:
c.execute('''drop table if exists distances''')

In [8]:
c.execute('''create table distances (
id integer, 
rel integer, 
ring integer)''')

In [9]:
c.execute('''create table model (
id integer,
iteration integer,
value real)
''')

ringnumbers is dict containing integer indexed string names of rings (of cvsf) {0:"strefa_0",1:"strefa_1",...}  

In [10]:
ringnumbers=dict([(int(fn.split(',')[0].split('_')[1]),fn) for fn in fieldnames if fn[:3]=="STR"])

id_name is string name of index field in cvsf

In [11]:
id_name=[fn for fn in fieldnames if fn[:2]=="ID"][0]

ids in ring example

In [12]:
csvf[7][ringnumbers[3]].split()

['20', '22']

Distance in distance table is ring number if 5 is in ring 10 of node 3 then we can say distance 3 to 5 is 10!

In [13]:
for row in csvf:
    id_n=row[id_name]
    for r in ringnumbers.keys():
        for rel in row[ringnumbers[r]].split():
            c.execute('''insert into distances values (?,?,?)''',(id_n,int(rel),r))

In [14]:
print(c.execute('''select count(*) from distances''').fetchone())

(544575,)


In [15]:
with open("DaneSIM.csv",'r') as cvsfile:
    f=csv.DictReader(cvsfile)
    fieldnamesd=f.fieldnames
    csvfd=[r for r in f]

cvsfd readed data
Each row is each region. Each contains data of iteration "Dest1":310 ,"Dest2":531...

iternumbers contains name of iteration indexed by number of iteration {0:"Dest0", 1: "Dest1"....

In [16]:
iternumbers=dict([(int(fn.split('_')[1]),fn) for fn in fieldnamesd if fn[:3]=="DES"])

Name of ID field 

In [17]:
id_named=[fn for fn in fieldnamesd if fn[:2]=="ID"][0];id_named

'ID'

### WARNING !!

Here we retrive reality from multilied REAL! observations.

In reality variable we store reality as dict indexed by  {0:0,1:0,2:1,3:0,4:1,5:2,6:0...

In [18]:
reality_column_name='pkt_archeo'
multiplier=7320

reality={row[id_named]:int(row[reality_column_name])//multiplier for row in csvfd}
assert sum(reality.values())==sum([int(row[reality_column_name]) for row in csvfd])//multiplier

In [19]:
c.execute('''drop table if exists reality''')

In [20]:
c.execute('''create table reality (
id integer,
value real)
''')

In [21]:
for k,v in reality.items():
    c.execute('''insert into reality values (?,?)''',(int(k),v))

In [22]:
c.execute('''delete from model''')

We store model values from each iteration. 

In [23]:
for row in csvfd:
    id_n=row[id_named]
    for it in iternumbers.keys():
        c.execute('''insert into model values (?,?,?)''',(id_n,it,float(row[iternumbers[it]])))

In [25]:
c.execute('''select count(*) from model''').fetchone()

(73800,)

In [26]:
len(csvfd)

738

In [27]:
c.execute('drop table if exists stodola')

This will be table of full stodołas

In [28]:
c.execute('''create table stodola (
id integer, 
iteration integer, 
noofstodola integer)''')

In [29]:
c.execute('drop table if exists stodola_part')

This will be list of not assigned parts of stodolas yet

In [30]:
c.execute('''create table stodola_part (
id integer,
iteration integer, 
stodolapart real)''')

In [31]:
c.execute('delete from stodola')

In [32]:
c.execute('delete from stodola_part')

Initially we want all values to be parts

In [33]:
c.execute('insert into stodola_part select * from model')

we have no full stodołas 

In [34]:
c.execute('insert into stodola select id,iteration,0 from model')

### How many stodoła we want to have finally

In [35]:
stodolas_number=386

How many modeled values for one (in tenth iteratio)  stodola (test only)

In [36]:
assert c.execute('''select sum(value) from reality''').fetchone()[0]==stodolas_number

In [37]:
c.execute('select sum(stodolapart) from stodola_part where iteration=?',[10]).fetchone()[0]/stodolas_number

6228.59067357513

In [38]:
c.close();c=dbconn.cursor()

ids is stored list of id

In [39]:
ids=[i[0] for i in c.execute('select distinct id from stodola')];

Function names says it all. Remaining parts sum of and the greatest part with id for iteration stated as parameter.

In [40]:
def remaining_parts(it):
    return c.execute('select sum(stodolapart) from stodola_part where iteration=?',[it]).fetchone()[0]
remaining_parts(10)

2404236.0

In [41]:
def bigest_stodola(it):
    return c.execute('select id,max(stodolapart) from stodola_part where iteration=?',[it]).fetchone()
bigest_stodola(1)

(454, 8443.0)

Following: some test and indexes nothing interesting here

In [42]:
c.execute('select max(ring) from distances where id=? ',[10]).fetchone()

(69,)

In [43]:
c.execute('select rel from distances where id=? and ring=?',[559,1]).fetchall()

[]

In [44]:
c.execute('''select sum(stodolapart) 
            from stodola_part 
            where iteration=? and 
            id in (select rel from distances  where id=? and ring=?)''',[1,559,0]).fetchone()

(6211.0,)

In [45]:
c.execute('create index if not exists distances_id_idx on distances (id)');
c.execute('create index if not exists distances_ring_idx on distances (ring)');

In [46]:
c.execute('create index if not exists stodola_id_idx on stodola (id)');
c.execute('create index if not exists stodola_iteration_idx on stodola (iteration)');
c.execute('create index if not exists stodola_noofstodola_idx on stodola (noofstodola)');

In [47]:
c.execute('create index if not exists stodola_part_id_idx on stodola_part (id)');
c.execute('create index if not exists stodola_part_iteration_idx on stodola_part (iteration)');
c.execute('create index if not exists stodola_part_noofstodola_idx on stodola_part (stodolapart)');

##### Fitnaess original 

fitness between reality and model

Model is divided so that it fits reality

We may say we use Euclid metrics but this can be changet into ex.: Marczewski - Steinhauss
We do not normalize as for now only relative chenges, in which model is better are, mesured
In other words this gives us relative not absolute model fitness

In [ ]:
for it in iternumbers.keys():
    print(it)

    with open('fitness_original.csv','w') as csvfitness:
        fields=[iternumbers[i+1] for i in range(it)]
        writer=csv.DictWriter(csvfitness,fieldnames=fields)
        writer.writeheader()
        
        row_dict=dict(c.execute('''
            select
                iteration,sum((model.value/?-reality.value)*(model.value/?-reality.value)) 
            from 
                model,
                reality 
            where
                model.id=reality.id and
                iteration<=?
            group by
                iteration''',[multiplier,multiplier,it]).fetchall())
        
        row_dict={iternumbers[k]:v**0.5 for k,v in row_dict.items()}
        writer.writerow(row_dict)     

#### Main part
## Stodolas algoritm

Warning it might be slow (about 1 iteration per 3 sec)


In [52]:
#prepare database
c.execute('delete from stodola')
c.execute('delete from stodola_part')
c.execute('insert into stodola_part select * from model')
c.execute('insert into stodola select id,iteration,0 from model')

#each iteration we consider separately
for it in iternumbers.keys():
    print(it)
    stodola_size=remaining_parts(it)/stodolas_number
    #extract whole stodolas
    idn,maks=bigest_stodola(it)
    while maks>=stodola_size:
        #keep integer part and fraction part
        noofstodola=int(maks//stodola_size)
        new_size=maks%stodola_size
        c.execute('update stodola_part set stodolapart=? where id=? and iteration=?',[new_size,idn,it])
        c.execute('update stodola set noofstodola=? where id=? and iteration=?',[noofstodola,idn,it])
        idn,maks=bigest_stodola(it)
    #extract bigest stodola 
    #here we have bigest stodola computed and corresponding id
    #last bigest migh not be strictly equal stodola_size (deal with numerical errors !)
    while remaining_parts(it)>0.9*stodola_size:
        stodola_to_find=stodola_size-maks
        #new full stodola
        c.execute('update stodola set noofstodola=noofstodola+1 where id=? and iteration=?',[idn,it])
        #stodola fully build no parts remains
        c.execute('update stodola_part set stodolapart=0 where id=? and iteration=?',[idn,it])
        max_strefa=c.execute('select max(ring) from distances where id=?',[idn]).fetchone()[0]
        #we keep reducucing each ring/strefa until stodola is fully biuld
        for ring in range(max_strefa):
            #we dont use ring 0 we use only 1..last+1
            #rels=[i[0] for i in c.execute('select rel from distances where id=? and ring=?',[idn,ring]).fetchall()]
            value_total_in_rels=c.execute('''select sum(stodolapart) 
            from stodola_part 
            where iteration=? and 
            id in (select rel from distances  where id=? and ring=?)''',[it,idn,ring+1]).fetchone()[0]
            if not value_total_in_rels:
                value_total_in_rels=0
            if value_total_in_rels>stodola_to_find:
                scale=(value_total_in_rels-stodola_to_find)/value_total_in_rels
                c.execute('''update stodola_part 
                set stodolapart=stodolapart*? 
                where iteration=? and 
                id in (select rel from distances  where id=? and ring=?)''',[scale,it,idn,ring+1])
                break
            else:
                c.execute('''update stodola_part 
                set stodolapart=0
                where iteration=? and 
                id in (select rel from distances  where id=? and ring=?)''',[it,idn,ring+1])
                stodola_to_find=stodola_to_find-value_total_in_rels
        idn,maks=bigest_stodola(it)

    
    # I create CSV after each iteration CSV contains stodolas for each iteration in data
    with open('stodoly.csv','w') as csvstodoly:
        fields=[id_named]+[iternumbers[i+1] for i in range(it)]
        writer=csv.DictWriter(csvstodoly,fieldnames=fields)
        writer.writeheader()
        for idn in ids:
            row_dict=dict(c.execute('select iteration,noofstodola from stodola where id=? and iteration<=?',[idn,it]).fetchall())
            row_dict={iternumbers[k]:v for k,v in row_dict.items()}           
            row_dict[id_named]=idn
            writer.writerow(row_dict)                

    # I create CSV after each iteration CSV contains geometric distance between REALITY and MODEL
    # We may say we use Euclid metrics but this can be changet into ex.: Marczewski - Steinhauss
    # We do not normalize as for now only relative chenges, in which model is better are, mesured
    # In other words this gives us relative not absolute model fitness
    with open('fitness.csv','w') as csvfitness:
        fields=[iternumbers[i+1] for i in range(it)]
        writer=csv.DictWriter(csvfitness,fieldnames=fields)
        writer.writeheader()
        
        row_dict=dict(c.execute('''
            select
                iteration,sum((stodola.noofstodola-reality.value)*(stodola.noofstodola-reality.value)) 
            from 
                stodola,
                reality 
            where
                stodola.id=reality.id and
                iteration<=?
            group by
                iteration''',[it]).fetchall())
        
        row_dict={iternumbers[k]:v**0.5 for k,v in row_dict.items()}
        writer.writerow(row_dict)                

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [87]:
c.execute('select * from stodola ').fetchall();

I tried to make visualisation (it wont work so just don't do following code)

import folium,json

map_osm = folium.Map(location=[50.2, 18.0],zoom_start=9)

gj=folium.GeoJson(open('./stodoly.geojson'),
                  style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': feature['properties']['stodoly_DEST_30'],
        'dashArray': '5, 5'
    }
                 ).add_to(map_osm)

folium.features.CircleMarker()